# 🤖 Gemma-2-2b-it 分類モデル訓練・保存ノートブック

## 概要
このノートブックは、MAP - Charting Student Math Misunderstandingsコンペティション用に、Gemma-2-2b-itモデルを6分類タスクでファインチューニングし、保存することを目的としています。

### モデル仕様
- **ベースモデル**: google/gemma-2-2b-it (~2.6B parameters)
- **タスク**: 6クラス分類 (True/False_Correct/Neither/Misconception)
- **ファインチューニング**: LoRA (Low-Rank Adaptation)
- **GPU**: A100 推奨

### 保存形式
- Google Drive: `/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/`
- ローカル: zipファイルでダウンロード
- 形式: Kaggle提出用のAutoModelForSequenceClassification

## 📦 環境セットアップ

In [1]:
# Google Driveマウント
from google.colab import drive
drive.mount('/content/drive')

# 必要なディレクトリ作成
import os
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/map_data', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/logs', exist_ok=True)

print("✅ Google Drive マウント完了")
print("✅ 必要ディレクトリ作成完了")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Google Drive マウント完了
✅ 必要ディレクトリ作成完了


In [2]:
# Hugging Face認証
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: read)

In [3]:
# 必要ライブラリのインストール/アップデート
!pip install peft accelerate transformers datasets -U -q
print("✅ ライブラリインストール/アップデート完了")

✅ ライブラリインストール/アップデート完了


## 📚 ライブラリとモデル設定

In [4]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from peft import LoraConfig, get_peft_model
import warnings
import json
import shutil
from pathlib import Path

warnings.filterwarnings("ignore")

# モデル設定
MODEL_NAME = "google/gemma-2-2b-it"
NUM_LABELS = 6
MAX_LENGTH = 512

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用デバイス: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print("✅ ライブラリ読み込み完了")

🖥️ 使用デバイス: cuda
🎮 GPU: NVIDIA A100-SXM4-40GB
💾 GPU メモリ: 42.5 GB
✅ ライブラリ読み込み完了


## 📊 データ読み込みと前処理

In [5]:
def load_and_prepare_data():
    """データの読み込みと前処理（6分類形式）"""
    print("=" * 60)
    print("📊 データ読み込みと前処理（6分類形式）")
    print("=" * 60)

    # データパス（Google Drive内）
    train_data_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/map_data/train.csv"

    if not os.path.exists(train_data_path):
        print(f"❌ データが見つかりません: {train_data_path}")
        print("💡 Google DriveのMyDrive/kaggleCompe_MAP-math/map_data/に train.csv をアップロードしてください")
        return None

    try:
        # データ読み込み
        train_df = pd.read_csv(train_data_path)
        print(f"✅ 訓練データ読み込み: {train_df.shape}")

        # コンペ形式のターゲット確認（6分類）
        print("\n📋 Category分布:")
        category_counts = train_df["Category"].value_counts()
        print(category_counts)

        # NaN値除去
        before_len = len(train_df)
        train_df = train_df.dropna(subset=["Category", "StudentExplanation"])
        after_len = len(train_df)
        print(f"\n🧹 NaN除去: {before_len} -> {after_len} ({before_len - after_len}行削除)")

        # 強化テキスト特徴量作成
        def create_enhanced_text(row):
            """Question + MC_Answer + Student Explanation を統合"""
            question = str(row["QuestionText"]) if pd.notna(row["QuestionText"]) else ""
            mc_answer = str(row["MC_Answer"]) if pd.notna(row["MC_Answer"]) else ""
            explanation = str(row["StudentExplanation"]) if pd.notna(row["StudentExplanation"]) else ""

            enhanced_text = f"Question: {question} Selected Answer: {mc_answer} Explanation: {explanation}"
            return enhanced_text

        print("\n🔧 強化テキスト特徴量作成中...")
        train_df["enhanced_text"] = train_df.apply(create_enhanced_text, axis=1)

        # 6つのカテゴリの確認
        unique_categories = sorted(train_df["Category"].unique())
        print(f"\n📊 ユニークなカテゴリ数: {len(unique_categories)}")
        print("カテゴリ一覧:")
        for i, cat in enumerate(unique_categories):
            count = (train_df["Category"] == cat).sum()
            percentage = count / len(train_df) * 100
            print(f"  {i}: {cat} ({count:,}件, {percentage:.1f}%)")

        # テキスト長統計
        text_lengths = train_df["enhanced_text"].str.len()
        print(f"\n📝 テキスト長統計:")
        print(f"  平均: {text_lengths.mean():.1f} 文字")
        print(f"  中央値: {text_lengths.median():.1f} 文字")
        print(f"  最大: {text_lengths.max()} 文字")
        print(f"  512文字以下: {(text_lengths <= 512).sum()} ({(text_lengths <= 512).mean()*100:.1f}%)")

        return train_df

    except Exception as e:
        print(f"❌ データ読み込みまたは前処理に失敗: {e}")
        return None

# データ読み込み実行
train_df = load_and_prepare_data()

📊 データ読み込みと前処理（6分類形式）
✅ 訓練データ読み込み: (36696, 7)

📋 Category分布:
Category
True_Correct           14802
False_Misconception     9457
False_Neither           6542
True_Neither            5265
True_Misconception       403
False_Correct            227
Name: count, dtype: int64

🧹 NaN除去: 36696 -> 36696 (0行削除)

🔧 強化テキスト特徴量作成中...

📊 ユニークなカテゴリ数: 6
カテゴリ一覧:
  0: False_Correct (227件, 0.6%)
  1: False_Misconception (9,457件, 25.8%)
  2: False_Neither (6,542件, 17.8%)
  3: True_Correct (14,802件, 40.3%)
  4: True_Misconception (403件, 1.1%)
  5: True_Neither (5,265件, 14.3%)

📝 テキスト長統計:
  平均: 222.1 文字
  中央値: 213.0 文字
  最大: 683 文字
  512文字以下: 36674 (99.9%)


## 🤖 Gemmaモデルとトークナイザー読み込み

In [6]:
def load_gemma_model(device):
    """Gemma-2-2b-itモデルとトークナイザーの読み込み（元の定義に準拠）"""
    print("\n" + "=" * 60)
    print(f"🤖 Gemmaモデル読み込み: {MODEL_NAME}")
    print("=" * 60)

    try:
        print("📝 Gemmaトークナイザー読み込み中...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

        # パディングトークンの設定（Gemmaの場合必要）
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("🔧 パディングトークンをEOSトークンに設定")

        print(f"✅ トークナイザー読み込み完了")
        print(f"🔖 パディングトークン: {tokenizer.pad_token}")
        print(f"📏 語彙サイズ: {tokenizer.vocab_size:,}")

        print("\n🧠 Gemmaモデル読み込み中...")

        # 分類タスク用の設定
        config = AutoConfig.from_pretrained(MODEL_NAME)
        config.num_labels = NUM_LABELS
        config.problem_type = "single_label_classification"

        # Gemma-2-2b-itモデルを分類用に読み込み
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            config=config,
            torch_dtype=torch.float16 if device.type == "cuda" else torch.float32,
            device_map="auto" if device.type == "cuda" else None,
        )

        print(f"✅ Gemmaモデル読み込み完了!")

        # モデル情報表示
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

        print(f"📊 分類クラス数: {NUM_LABELS}")
        print(f"📈 総パラメータ数: {total_params:,}")
        print(f"🎯 訓練可能パラメータ数: {trainable_params:,}")
        print(f"💡 モデルサイズ: ~{total_params / 1e9:.2f}B parameters")

        return model, tokenizer

    except Exception as e:
        print(f"❌ モデル読み込み失敗: {e}")
        return None, None

# モデル読み込み実行
model, tokenizer = load_gemma_model(device)


🤖 Gemmaモデル読み込み: google/gemma-2-2b-it
📝 Gemmaトークナイザー読み込み中...
✅ トークナイザー読み込み完了
🔖 パディングトークン: <pad>
📏 語彙サイズ: 256,000

🧠 Gemmaモデル読み込み中...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of Gemma2ForSequenceClassification were not initialized from the model checkpoint at google/gemma-2-2b-it and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✅ Gemmaモデル読み込み完了!
📊 分類クラス数: 6
📈 総パラメータ数: 2,614,355,712
🎯 訓練可能パラメータ数: 2,614,355,712
💡 モデルサイズ: ~2.61B parameters


## 🔧 LoRA設定とPEFT適用

In [7]:
def apply_lora_to_model(model):
    """LoRA設定とPEFT適用"""
    print("\n" + "=" * 60)
    print("🔧 LoRA (PEFT) 設定と適用")
    print("=" * 60)

    if model is None:
        print("❌ ベースモデルが準備されていません")
        return None

    try:
        # PEFTライブラリのバージョン確認
        import peft
        print(f"📦 PEFTライブラリバージョン: {peft.__version__}")

        # PEFT適用前にモデルのデータ型を明示的にfloat32にする
        print("🔄 PEFT適用前にモデルをfloat32にキャスト...")
        model = model.to(torch.float32)
        print(f"✅ モデルデータ型: {next(model.parameters()).dtype}")


        # LoRA設定
        lora_config = LoraConfig(
            r=16,                    # LoRA attention dimension
            lora_alpha=32,          # Alpha parameter for LoRA scaling
            target_modules=[        # Gemma-2の主要な線形層をターゲット
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
            lora_dropout=0.05,      # Dropout probability for LoRA layers
            bias="none",            # Bias type
            task_type="SEQ_CLS",    # Sequence Classification task
            modules_to_save=["classifier", "score"],  # 分類ヘッドを保存
        )

        print("📋 LoRA設定:")
        print(f"  r (rank): {lora_config.r}")
        print(f"  lora_alpha: {lora_config.lora_alpha}")
        print(f"  target_modules: {lora_config.target_modules}")
        print(f"  lora_dropout: {lora_config.lora_dropout}")
        print(f"  task_type: {lora_config.task_type}")

        # PEFTモデル作成
        print("\n🔄 PEFTモデル作成中...")
        peft_model = get_peft_model(model, lora_config)

        print("✅ PEFTモデル作成完了")
        peft_model.print_trainable_parameters()

        # デバイスに移動
        # PEFT適用後、モデルを再度デバイスに移動 (get_peft_modelがCPUに戻す場合があるため)
        print(f"📍 PEFTモデルをデバイスに移動: {device}")
        peft_model.to(device)
        print(f"✅ モデルがデバイスに移動しました: {next(peft_model.parameters()).device}")


        return peft_model

    except Exception as e:
        print(f"❌ LoRA適用失敗: {e}")
        return None

# LoRA適用
if model is not None:
    peft_model = apply_lora_to_model(model)
else:
    print("❌ モデルが読み込まれていないため、LoRAを適用できません")
    peft_model = None


🔧 LoRA (PEFT) 設定と適用
📦 PEFTライブラリバージョン: 0.16.0
🔄 PEFT適用前にモデルをfloat32にキャスト...
✅ モデルデータ型: torch.float32
📋 LoRA設定:
  r (rank): 16
  lora_alpha: 32
  target_modules: {'down_proj', 'gate_proj', 'o_proj', 'k_proj', 'up_proj', 'v_proj', 'q_proj'}
  lora_dropout: 0.05
  task_type: SEQ_CLS

🔄 PEFTモデル作成中...
✅ PEFTモデル作成完了
trainable params: 20,780,544 || all params: 2,635,136,256 || trainable%: 0.7886
📍 PEFTモデルをデバイスに移動: cuda
✅ モデルがデバイスに移動しました: cuda:0


## 📝 データセットクラス定義

In [8]:
class MathMisconceptionDataset(Dataset):
    """Math Misconception Dataset for PyTorch"""

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # トークン化
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

print("✅ MathMisconceptionDataset クラス定義完了")

✅ MathMisconceptionDataset クラス定義完了


## 📊 評価メトリクス定義

In [9]:
def compute_map3_metrics(eval_pred):
    """MAP@3メトリクスの計算"""
    predictions, labels = eval_pred
    predictions = torch.softmax(torch.tensor(predictions), dim=-1).numpy()

    map_scores = []
    for i, true_label in enumerate(labels):
        top3_indices = np.argsort(predictions[i])[::-1][:3]
        score = 0.0
        for j, pred_idx in enumerate(top3_indices):
            if pred_idx == true_label:
                score = 1.0 / (j + 1)
                break
        map_scores.append(score)

    map3_score = np.mean(map_scores)
    accuracy = accuracy_score(labels, np.argmax(predictions, axis=1))

    return {"map3": map3_score, "accuracy": accuracy}

print("✅ MAP@3評価メトリクス定義完了")

✅ MAP@3評価メトリクス定義完了


## 🚀 モデル訓練実行

In [10]:
def train_model(train_df, peft_model, tokenizer, device):
    """モデルのファインチューニング実行"""
    print("\n" + "=" * 60)
    print("🚀 6分類モデル ファインチューニング開始")
    print("=" + "=" * 60)

    if train_df is None or peft_model is None:
        print("❌ 訓練データまたはモデルが準備されていません")
        return None, None

    # ラベルエンコーディング
    label_encoder = LabelEncoder()
    train_df["encoded_labels"] = label_encoder.fit_transform(train_df["Category"])

    print(f"📋 エンコードされたラベル:")
    for i, label in enumerate(label_encoder.classes_):
        count = (train_df["encoded_labels"] == i).sum()
        print(f"  {i}: {label} ({count:,}件)")

    # 訓練/検証分割
    train_texts = train_df["enhanced_text"].tolist()
    train_labels = train_df["encoded_labels"].tolist()

    try:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts,
            train_labels,
            test_size=0.2,
            random_state=42,
            stratify=train_labels,
        )
        print("✅ Stratified split適用")
    except ValueError:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts, train_labels, test_size=0.2, random_state=42
        )
        print("✅ Regular split適用（少数クラスのため）")

    print(f"📊 訓練データ: {len(X_train):,}件")
    print(f"📊 検証データ: {len(X_val):,}件")

    # データセット作成
    train_dataset = MathMisconceptionDataset(
        X_train, y_train, tokenizer, max_length=MAX_LENGTH
    )
    val_dataset = MathMisconceptionDataset(
        X_val, y_val, tokenizer, max_length=MAX_LENGTH
    )

    # 訓練設定 - おすすめ設定を適用
    training_args = TrainingArguments(
        output_dir="/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification",
        num_train_epochs=3, # おすすめ設定: 3エポック
        per_device_train_batch_size=4,  # おすすめ設定: バッチサイズ 4
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=8,  # おすすめ設定: 実効バッチサイズ 32
        warmup_steps=200,
        weight_decay=0.01,
        logging_dir="/content/drive/MyDrive/kaggleCompe_MAP-math/logs",
        logging_steps=100,
        eval_strategy="steps",
        eval_steps=300,
        save_strategy="steps",
        save_steps=300,
        load_best_model_at_end=True,
        metric_for_best_model="map3",
        greater_is_better=True,
        report_to="none",
        dataloader_pin_memory=False,
        fp16=False,
        bf16=True, # おすすめ設定: bf16を有効化
        optim="adamw_torch",
        learning_rate=2e-5,
        save_total_limit=2,
        gradient_checkpointing=False,
    )

    print("\n📋 訓練設定:")
    print(f"  エポック数: {training_args.num_train_epochs}")
    print(f"  per_device_train_batch_size: {training_args.per_device_train_batch_size}")
    print(f"  gradient_accumulation_steps: {training_args.gradient_accumulation_steps}")
    print(f"  実効バッチサイズ: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
    print(f"  学習率: {training_args.learning_rate}")
    print(f"  保存先: {training_args.output_dir}")
    print(f"  bf16: {training_args.bf16}")


    # Trainer作成
    trainer = Trainer(
        model=peft_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_map3_metrics,
    )

    # 訓練実行
    print("\n🚀 ファインチューニング開始...")
    try:
        trainer.train()
        print("✅ ファインチューニング完了")

        # 最終評価
        print("\n📊 最終評価:")
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            print(f"  {key}: {value:.4f}")

        return trainer, label_encoder

    except Exception as e:
        print(f"❌ ファインチューニング中にエラー: {e}")
        return None, None

# 訓練実行
if train_df is not None and peft_model is not None:
    trainer, label_encoder = train_model(train_df, peft_model, tokenizer, device)
else:
    print("❌ 訓練に必要な要素が準備されていません")
    trainer, label_encoder = None, None

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.



🚀 6分類モデル ファインチューニング開始
📋 エンコードされたラベル:
  0: False_Correct (227件)
  1: False_Misconception (9,457件)
  2: False_Neither (6,542件)
  3: True_Correct (14,802件)
  4: True_Misconception (403件)
  5: True_Neither (5,265件)
✅ Stratified split適用
📊 訓練データ: 29,356件
📊 検証データ: 7,340件

📋 訓練設定:
  エポック数: 3
  per_device_train_batch_size: 4
  gradient_accumulation_steps: 8
  実効バッチサイズ: 32
  学習率: 2e-05
  保存先: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification
  bf16: True

🚀 ファインチューニング開始...


Step,Training Loss,Validation Loss,Map3,Accuracy
300,0.752000,0.559650,0.878156,0.767030
600,0.453000,0.461666,0.901317,0.809264
900,0.383600,0.391792,0.919482,0.845368
1200,0.322600,0.354655,0.928588,0.861172
1500,0.296100,0.345375,0.930313,0.864986
1800,0.277000,0.330186,0.933356,0.870845
2100,0.197100,0.356158,0.934718,0.873569
2400,0.175100,0.376239,0.935059,0.874251
2700,0.175000,0.357493,0.935831,0.875749


✅ ファインチューニング完了

📊 最終評価:


  eval_loss: 0.3575
  eval_map3: 0.9358
  eval_accuracy: 0.8757
  eval_runtime: 223.7793
  eval_samples_per_second: 32.8000
  eval_steps_per_second: 8.2000
  epoch: 3.0000


## 💾 モデル保存

In [11]:
def save_trained_model(trainer, tokenizer, label_encoder):
    """訓練済みモデルの保存（Kaggle提出用形式）"""
    print("\n" + "=" * 60)
    print("💾 訓練済みモデル保存")
    print("=" * 60)

    if trainer is None or tokenizer is None or label_encoder is None:
        print("❌ 保存に必要な要素が準備されていません")
        return False

    save_base_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models"
    model_save_path = f"{save_base_path}/gemma-2-2b-math-classification-final"

    try:
        # モデルとトークナイザーの保存
        print(f"📁 保存先: {model_save_path}")

        # トレーナーからモデルを保存（PEFTモデル込み）
        trainer.save_model(model_save_path)
        tokenizer.save_pretrained(model_save_path)

        print("✅ モデルとトークナイザー保存完了")

        # ラベルマッピングの保存
        label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
        label_file = f"{model_save_path}/label_mapping.json"

        with open(label_file, "w", encoding="utf-8") as f:
            json.dump(label_mapping, f, ensure_ascii=False, indent=2)

        print("✅ ラベルマッピング保存完了")
        print(f"📋 ラベルマッピング: {label_mapping}")

        # メタデータの保存
        metadata = {
            "model_name": MODEL_NAME,
            "num_labels": NUM_LABELS,
            "max_length": MAX_LENGTH,
            "task_type": "sequence_classification",
            "architecture": "AutoModelForSequenceClassification",
            "peft_applied": True,
            "lora_config": {
                "r": 16,
                "lora_alpha": 32,
                "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                "task_type": "SEQ_CLS"
            }
        }

        metadata_file = f"{model_save_path}/model_metadata.json"
        with open(metadata_file, "w", encoding="utf-8") as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)

        print("✅ メタデータ保存完了")

        # 保存されたファイルの確認
        print(f"\n📂 保存されたファイル:")
        for item in os.listdir(model_save_path):
            item_path = os.path.join(model_save_path, item)
            if os.path.isfile(item_path):
                size_mb = os.path.getsize(item_path) / (1024 * 1024)
                print(f"  📄 {item}: {size_mb:.2f} MB")
            else:
                print(f"  📁 {item}/")

        return True

    except Exception as e:
        print(f"❌ モデル保存中にエラー: {e}")
        return False

# モデル保存実行
if trainer is not None and tokenizer is not None and label_encoder is not None:
    save_success = save_trained_model(trainer, tokenizer, label_encoder)

    if save_success:
        print("\n🎉 モデル保存完了!")
        print("📁 Google Drive保存パス: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/")
        print("🚀 次のステップ: Kaggleデータセットとしてアップロード準備完了")
    else:
        print("❌ モデル保存に失敗しました")
else:
    print("❌ 保存に必要な要素が準備されていません")


💾 訓練済みモデル保存
📁 保存先: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification-final
✅ モデルとトークナイザー保存完了
✅ ラベルマッピング保存完了
📋 ラベルマッピング: {0: 'False_Correct', 1: 'False_Misconception', 2: 'False_Neither', 3: 'True_Correct', 4: 'True_Misconception', 5: 'True_Neither'}
✅ メタデータ保存完了

📂 保存されたファイル:
  📄 README.md: 0.00 MB
  📄 adapter_model.safetensors: 79.32 MB
  📄 adapter_config.json: 0.00 MB
  📄 chat_template.jinja: 0.00 MB
  📄 tokenizer_config.json: 0.04 MB
  📄 special_tokens_map.json: 0.00 MB
  📄 tokenizer.model: 4.04 MB
  📄 tokenizer.json: 32.77 MB
  📄 training_args.bin: 0.01 MB
  📄 label_mapping.json: 0.00 MB
  📄 model_metadata.json: 0.00 MB

🎉 モデル保存完了!
📁 Google Drive保存パス: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/
🚀 次のステップ: Kaggleデータセットとしてアップロード準備完了


## 📦 ローカルダウンロード用zip作成

In [12]:
def create_download_zip():
    """ローカルダウンロード用のzipファイル作成"""
    print("\n" + "=" * 60)
    print("📦 ローカルダウンロード用zip作成")
    print("=" * 60)

    try:
        import zipfile

        model_dir = "/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification-final"
        zip_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip"

        if not os.path.exists(model_dir):
            print(f"❌ モデルディレクトリが見つかりません: {model_dir}")
            return False

        print(f"📁 ソース: {model_dir}")
        print(f"📦 出力zip: {zip_path}")

        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(model_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, model_dir)
                    zipf.write(file_path, arcname)
                    print(f"  ➕ {arcname}")

        # zipファイルサイズ確認
        zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"\n✅ zip作成完了!")
        print(f"📏 zipサイズ: {zip_size_mb:.2f} MB")
        print(f"📥 ダウンロードパス: {zip_path}")

        # ダウンロード用コード表示
        print(f"\n💡 Colabでダウンロードするには:")
        print(f"```python")
        print(f"from google.colab import files")
        print(f"files.download('{zip_path}')")
        print(f"```")

        return True

    except Exception as e:
        print(f"❌ zip作成中にエラー: {e}")
        return False

# zip作成実行
zip_success = create_download_zip()


📦 ローカルダウンロード用zip作成
📁 ソース: /content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/gemma-2-2b-math-classification-final
📦 出力zip: /content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip
  ➕ README.md
  ➕ adapter_model.safetensors
  ➕ adapter_config.json
  ➕ chat_template.jinja
  ➕ tokenizer_config.json
  ➕ special_tokens_map.json
  ➕ tokenizer.model
  ➕ tokenizer.json
  ➕ training_args.bin
  ➕ label_mapping.json
  ➕ model_metadata.json

✅ zip作成完了!
📏 zipサイズ: 80.58 MB
📥 ダウンロードパス: /content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip

💡 Colabでダウンロードするには:
```python
from google.colab import files
files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')
```


## 🎯 完了サマリー

### ✅ 完了タスク
1. **🤖 モデル読み込み**: Gemma-2-2b-it (AutoModelForSequenceClassification)
2. **🔧 LoRA適用**: 効率的なファインチューニング
3. **📊 データ準備**: 6分類タスク用前処理
4. **🚀 訓練実行**: MAP@3最適化
5. **💾 モデル保存**: Google Drive + zip形式

### 📁 保存場所
- **Google Drive**: `/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/`
- **zip形式**: `/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip`

### 🔄 次のステップ
1. **Kaggleデータセット作成**: 保存されたモデルをKaggleにアップロード
2. **推論ノートブック**: test_saved2-2b.ipynbで読み込みテスト
3. **提出**: submission.csvの生成

### 🎯 Kaggle提出用読み込み形式
```python
model = AutoModelForSequenceClassification.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)
```

**🎉 Gemma-2-2b-it分類モデル訓練完了!**

In [13]:
# 最終実行確認とダウンロード
if zip_success:
    print("🎉 すべての処理が完了しました!")
    print("\n📥 モデルをダウンロードするには以下を実行:")
    print("```python")
    print("from google.colab import files")
    print("files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')")
    print("```")

    # 自動ダウンロード（オプション）
    try:
        from google.colab import files
        response = input("今すぐダウンロードしますか？ (y/n): ")
        if response.lower() == 'y':
            files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')
            print("✅ ダウンロード開始!")
    except:
        print("💡 手動でダウンロードしてください")
else:
    print("❌ 一部の処理が完了していません。上記のセルを確認してください。")

🎉 すべての処理が完了しました!

📥 モデルをダウンロードするには以下を実行:
```python
from google.colab import files
files.download('/content/drive/MyDrive/kaggleCompe_MAP-math/gemma-2-2b-math-model.zip')
```
今すぐダウンロードしますか？ (y/n): y


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ ダウンロード開始!


訓練ができないエラーの調査用だから終わったらいらないコード

In [14]:
# PEFT適用後のモデル構造とtrainableパラメータの確認
if peft_model is not None:
    print("\n" + "=" * 60)
    print("🧠 PEFT適用後のモデル構造とTrainableパラメータ:")
    print("=" * 60)

    # 分類ヘッドと思われる層の名前を特定
    # Gemma2ForSequenceClassificationの構造を確認し、分類ヘッドを探す
    classifier_names = ["score", "classifier", "lm_head"] # よく使われる名前候補

    found_classifier = False
    for name, module in peft_model.named_modules():
        is_target_classifier = False
        for cls_name in classifier_names:
            if cls_name in name:
                is_target_classifier = True
                break

        if "lora" in name:
             # LoRA層はtrainableになっているはず
             print(f"✅ LoRA Layer: {name} (requires_grad: {module.weight.requires_grad if hasattr(module, 'weight') else False})")
        elif is_target_classifier and isinstance(module, (torch.nn.Linear, torch.nn.ModuleDict, torch.nn.Sequential)):
             # 分類ヘッドと思われる層の情報
             print(f"\n🎯 Classifier/Score Layer: {name}")
             print(f"  Type: {type(module)}")
             # 再帰的にサブモジュールのrequires_gradを確認
             all_classifier_params_trainable = True
             classifier_params_count = 0
             for param_name, param in module.named_parameters():
                 print(f"    - {name}.{param_name} (requires_grad: {param.requires_grad})")
                 if not param.requires_grad:
                     all_classifier_params_trainable = False
                 classifier_params_count += param.numel()
             print(f"  Total parameters in this module: {classifier_params_count:,}")
             print(f"  All parameters trainable in this module: {all_classifier_params_trainable}")
             found_classifier = True
        elif any(target_m in name for target_m in ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"]):
             # LoRAが適用されているベースモデルの線形層（自身は凍結されているはず）
             pass # 詳細な出力は省略
        else:
             # その他の層（通常は凍結）
             pass # 詳細な出力は省略

    if not found_classifier:
        print("\n⚠️ 警告: 分類ヘッド ('score' or 'classifier') がモデル構造内で見つかりませんでした。")
        print("PEFTの設定 'modules_to_save' の名前を確認してください。")

    # PEFTモデル全体の trainable パラメータを再確認 (念のため)
    total_trainable_params = sum(p.numel() for p in peft_model.parameters() if p.requires_grad)
    print(f"\n📈 PEFTモデル全体の訓練可能パラメータ数: {total_trainable_params:,}")

else:
    print("❌ PEFTモデルが作成されていません。")


🧠 PEFT適用後のモデル構造とTrainableパラメータ:
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_dropout (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_dropout.default (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_A (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_A.default (requires_grad: True)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_B (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_B.default (requires_grad: True)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_A (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_B (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.q_proj.lora_magnitude_vector (requires_grad: False)
✅ LoRA Layer: base_model.model.model.layers.0.self_attn.k_pro

In [15]:
# PEFT適用後のモデル重みの確認
if peft_model is not None:
    print("\n" + "=" * 60)
    print("🧠 PEFT適用後のモデル重み確認:")
    print("=" + "=" * 60)

    # 分類ヘッド (score) の重みを確認
    score_layer = None
    for name, module in peft_model.named_modules():
        if name == "score" and isinstance(module, torch.nn.Linear):
            score_layer = module
            break

    if score_layer:
        print(f"\n🎯 分類ヘッド ('score') の重み ({score_layer.weight.shape}):")
        weight = score_layer.weight.data.cpu().numpy()
        print(f"  平均: {np.mean(weight):.6f}")
        print(f"  標準偏差: {np.std(weight):.6f}")
        print(f"  最小値: {np.min(weight):.6f}")
        print(f"  最大値: {np.max(weight):.6f}")
        # ゼロが含まれていないか軽くチェック
        zero_count = np.sum(weight == 0)
        total_count = weight.size
        print(f"  ゼロ値の数: {zero_count} ({zero_count / total_count * 100:.4f}%)")
    else:
        print("\n⚠️ 警告: 分類ヘッド ('score') レイヤーが見つかりませんでした。")

    # 一部のLoRA層の重みを確認 (例: 最初のAttention Q_projのAとB)
    lora_A_weight = None
    lora_B_weight = None
    lora_layer_found = False
    for name, module in peft_model.named_modules():
        if "layers.0.self_attn.q_proj.lora_A.default" in name and hasattr(module, 'weight'):
             lora_A_weight = module.weight.data.cpu().numpy()
             lora_layer_found = True
             print(f"\n✅ LoRA-A ({name}) の重み ({module.weight.shape}):")
             print(f"  平均: {np.mean(lora_A_weight):.6f}")
             print(f"  標準偏差: {np.std(lora_A_weight):.6f}")
             print(f"  最小値: {np.min(lora_A_weight):.6f}")
             print(f"  最大値: {np.max(lora_A_weight):.6f}")
             zero_count = np.sum(lora_A_weight == 0)
             total_count = lora_A_weight.size
             print(f"  ゼロ値の数: {zero_count} ({zero_count / total_count * 100:.4f}%)")

        if "layers.0.self_attn.q_proj.lora_B.default" in name and hasattr(module, 'weight'):
             lora_B_weight = module.weight.data.cpu().numpy()
             print(f"\n✅ LoRA-B ({name}) の重み ({module.weight.shape}):")
             print(f"  平均: {np.mean(lora_B_weight):.6f}")
             print(f"  標準偏差: {np.std(lora_B_weight):.6f}")
             print(f"  最小値: {np.min(lora_B_weight):.6f}")
             print(f"  最大値: {np.max(lora_B_weight):.6f}")
             zero_count = np.sum(lora_B_weight == 0)
             total_count = lora_B_weight.size
             print(f"  ゼロ値の数: {zero_count} ({zero_count / total_count * 100:.4f}%)")

        # 他のLoRA層も確認したい場合はここに追加

    if not lora_layer_found:
         print("\n⚠️ 警告: LoRA層の重みが見つかりませんでした。PEFTが正しく適用されていない可能性があります。")

else:
    print("❌ PEFTモデルが作成されていません。")


🧠 PEFT適用後のモデル重み確認:

⚠️ 警告: 分類ヘッド ('score') レイヤーが見つかりませんでした。

✅ LoRA-A (base_model.model.model.layers.0.self_attn.q_proj.lora_A.default) の重み (torch.Size([16, 2304])):
  平均: 0.000020
  標準偏差: 0.012074
  最小値: -0.022465
  最大値: 0.022262
  ゼロ値の数: 0 (0.0000%)

✅ LoRA-B (base_model.model.model.layers.0.self_attn.q_proj.lora_B.default) の重み (torch.Size([2048, 16])):
  平均: -0.000007
  標準偏差: 0.000564
  最小値: -0.002245
  最大値: 0.002472
  ゼロ値の数: 0 (0.0000%)
